# PyTorchでの学習・推論を高速化するコツ集
## MNISTでいろいろ試す
### 初期設定


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# pytorch version 確認
torch.__version__  # 1.6.0+cu101


'1.6.0+cu101'

In [3]:
# GPU使用の確認
# Google Colaboratoryでは「ランタイム」→「ランタイムタイムを変更」でGPUに

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # cuda(GPU)


cuda


In [4]:
# GPUの確認
!nvidia-smi

Tue Sep  8 21:03:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### ネットワーク・モデルの設定

In [5]:
# 参考: https://github.com/pytorch/examples/blob/master/mnist/main.py

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
        

## データセットと前処理の設定


In [6]:
# 前処理
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

# 訓練データ
dataset1 = datasets.MNIST('.', train=True, download=True,
                    transform=transform)

# 検証データ
dataset2 = datasets.MNIST('.', train=False,
                    transform=transform)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 訓練と検証の関数作成

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # 訓練モードに
    for batch_idx, (data, target) in enumerate(train_loader):
        # データ取り出し
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()

        # 伝搬
        output = model(data)
        
        # 損失計算とバックプロパゲーション
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [8]:
def test(model, device, test_loader):
    model.eval()  # 検証モードに
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # データ取り出し
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## 1. DataLoaderについて

In [9]:
# CPUのコア数を確認
import os
os.cpu_count()  # コア


2

In [10]:
# 関数化
import time

def MNIST_train(optimizer, model, device, train_loader, test_loader): 
    # デフォルトで訓練
    epochs = 1

    start = time.time()

    # 処理
    for epoch in range(1, epochs+1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

    # かかった時間
    print("=======かかった時間========")
    print(time.time() - start)


### 1.1.1 デフォルト

In [11]:
# ミニバッチのサイズ
mini_batch_size = 512

In [12]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [13]:
# デフォルト設定のDataLoaderの場合
train_loader_default = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size)
test_loader_default = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size)


In [14]:
MNIST_train(optimizer, model, device, train_loader_default, test_loader_default)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306127
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.328438

Test set: Average loss: 0.2787, Accuracy: 9161/10000 (92%)

=======かかった時間========
8.479081630706787


### 1.1.2 DataLoaderの引数num_workersを設定

In [15]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [16]:
# データローダー
train_loader_nworker = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=os.cpu_count()) 
test_loader_nworker = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=os.cpu_count())

In [17]:
MNIST_train(optimizer, model, device, train_loader_nworker, test_loader_nworker)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306503
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.296145

Test set: Average loss: 0.2640, Accuracy: 9198/10000 (92%)

=======かかった時間========
8.013897180557251


### 1.1.3 DataLoaderの引数pin_memoryをTrueに設定

In [18]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [19]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, pin_memory=True)

In [20]:
MNIST_train(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300168
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.378180

Test set: Average loss: 0.3052, Accuracy: 9050/10000 (90%)

=======かかった時間========
8.52079963684082


In [21]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True)

In [22]:
MNIST_train(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.399404
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.217007

Test set: Average loss: 0.2028, Accuracy: 9357/10000 (94%)

=======かかった時間========
8.253820180892944


## 2. torch.backends.cudnn.benchmark = True

In [23]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [24]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=False) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=False)

In [25]:
# 関数化

def MNIST_train_cudnn_benchmark_True(optimizer, model, device, train_loader, test_loader): 
    # デフォルトで訓練
    epochs = 1

    start = time.time()
    
    # 追加
    torch.backends.cudnn.benchmark = True

    # 処理
    for epoch in range(1, epochs+1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

    # かかった時間
    print("=======かかった時間========")
    print(time.time() - start)
    

In [26]:
MNIST_train_cudnn_benchmark_True(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.317815
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.305686

Test set: Average loss: 0.2599, Accuracy: 9198/10000 (92%)

=======かかった時間========
8.410441160202026


In [27]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [28]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True)

In [29]:
MNIST_train_cudnn_benchmark_True(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.314690
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.288278

Test set: Average loss: 0.2991, Accuracy: 9075/10000 (91%)

=======かかった時間========
8.079831838607788


## JITで単純な計算

In [30]:
x = torch.randn(2000, 30, 200)


In [31]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / 1.41421))

In [32]:
start = time.time()

for i in range(200):
    gelu(x)

# かかった時間
print("=======かかった時間========")
print(time.time() - start)


=======かかった時間========
12.339202880859375


In [33]:
@torch.jit.script
def fused_gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / 1.41421))

In [34]:
start = time.time()

for i in range(200):
    fused_gelu(x)
    
# かかった時間
print("=======かかった時間========")
print(time.time() - start)

=======かかった時間========
12.223289966583252


## PyTorch AMP

In [35]:
# https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples

In [36]:
def train_PyTorchAMP(model, device, train_loader, optimizer, epoch):
    model.train()  # 訓練モードに
    
    scaler = torch.cuda.amp.GradScaler()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        # データ取り出し
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()

        # 伝搬
        # Runs the forward pass with autocasting.
        with torch.cuda.amp.autocast():
            output = model(data)
            loss = F.nll_loss(output, target)
        
        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        scaler.scale(loss).backward()

        
        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(optimizer)

        # Updates the scale for next iteration.
        scaler.update()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [37]:
# 関数化

def MNIST_train_PyTorchAMP(optimizer, model, device, train_loader, test_loader): 
    # デフォルトで訓練
    epochs = 1
    
    start = time.time()
    
    # 追加
    torch.backends.cudnn.benchmark = True

    # 処理
    for epoch in range(1, epochs+1):
        train_PyTorchAMP(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

    # かかった時間
    print("=======かかった時間========")
    print(time.time() - start)

In [38]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)


In [39]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)


In [40]:
MNIST_train_PyTorchAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300802
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.360303

Test set: Average loss: 0.3280, Accuracy: 8950/10000 (90%)

=======かかった時間========
10.011407136917114


In [41]:
#

In [42]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)


In [43]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True)

In [44]:
MNIST_train_PyTorchAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308190
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.317418

Test set: Average loss: 0.3194, Accuracy: 8946/10000 (89%)

=======かかった時間========
8.474640607833862


## non_blocking=True

In [50]:
def train_non_blocking_true(model, device, train_loader, optimizer, epoch):
    model.train()  # 訓練モードに
    for batch_idx, (data, target) in enumerate(train_loader):
        # データ取り出し
        data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
        
        optimizer.zero_grad()

        # 伝搬
        output = model(data)
        
        # 損失計算とバックプロパゲーション
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [51]:
def test_non_blocking_true(model, device, test_loader):
    model.eval()  # 検証モードに
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # データ取り出し
            data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [52]:
# 関数化

def MNIST_train_non_blocking_true(optimizer, model, device, train_loader, test_loader): 
    # デフォルトで訓練
    epochs = 1

    start = time.time()
    
    # 追加
    torch.backends.cudnn.benchmark = True

    # 処理
    for epoch in range(1, epochs+1):
        train_non_blocking_true(model, device, train_loader, optimizer, epoch)
        test_non_blocking_true(model, device, test_loader)

    # かかった時間
    print("=======かかった時間========")
    print(time.time() - start)
    

In [76]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)


In [77]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)

In [78]:
MNIST_train_non_blocking_true(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312252
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.393690

Test set: Average loss: 0.3092, Accuracy: 9045/10000 (90%)

=======かかった時間========
8.370294570922852


In [79]:
# default

In [80]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)


In [81]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)

In [82]:
MNIST_train(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307320
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.345337

Test set: Average loss: 0.3234, Accuracy: 9001/10000 (90%)

=======かかった時間========
8.298707246780396


# APX

In [45]:
# 以下は
# https://github.com/NVIDIA/apex
# を参考にAPXをインストールしておく

#$ git clone https://github.com/NVIDIA/apex
#$ cd apex
#s$ pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [46]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [47]:
from apex import amp, optimizers

# Initialization
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


ModuleNotFoundError: ignored

In [ ]:
def trainAMP(model, device, train_loader, optimizer, epoch):
    model.train()  # 訓練モードに
        
    for batch_idx, (data, target) in enumerate(train_loader):
        # データ取り出し
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()

        # 伝搬
        output = model(data)
        
        # 損失計算とバックプロパゲーション
        loss = F.nll_loss(output, target)
        
        # AMP Train your model
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [ ]:
# 関数化

def MNIST_trainAMP(optimizer, model, device, train_loader, test_loader): 
    # デフォルトで訓練
    epochs = 1

    start = time.time()
    
    # 追加
    torch.backends.cudnn.benchmark = True

    # 処理
    for epoch in range(1, epochs+1):
        trainAMP(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

    # かかった時間
    print("=======かかった時間========")
    print(time.time() - start)
    

In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)

In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)

In [ ]:
# 

In [ ]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [ ]:
from apex import amp, optimizers

# Initialization
opt_level = 'O2'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)

In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)

In [ ]:
#

In [ ]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [ ]:
from apex import amp, optimizers

# Initialization
opt_level = 'O3'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)


In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


In [ ]:
#

In [ ]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [ ]:
from apex import amp, optimizers

# Initialization
opt_level = 'O0'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)


In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


In [ ]:
#

In [ ]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = optim.Adadelta(model.parameters(), lr=lr_rate)

In [ ]:
from apex import amp, optimizers

# Initialization
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True)


In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


## APEX

In [ ]:
import apex

# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = apex.optimizers.FusedSGD(model.parameters(), lr=lr_rate)

In [ ]:
from apex import amp, optimizers

# Initialization
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)


In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


In [ ]:
#

In [ ]:
import apex

# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = apex.optimizers.FusedLAMB(model.parameters(), lr=lr_rate)


In [ ]:
from apex import amp, optimizers

# Initialization
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=0, pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=0, pin_memory=True)


In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


In [ ]:
#

In [ ]:
# モデル、学習率とoptimizerを設定
model = Net().to(device)
lr_rate = 0.1
optimizer = apex.optimizers.FusedSGD(model.parameters(), lr=lr_rate)

In [ ]:
from apex import amp, optimizers

# Initialization
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)


In [ ]:
# データローダー
train_loader_pin_memory = torch.utils.data.DataLoader(dataset1,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True) 
test_loader_pin_memory = torch.utils.data.DataLoader(dataset2,batch_size=mini_batch_size, num_workers=os.cpu_count(), pin_memory=True)

In [ ]:
MNIST_trainAMP(optimizer, model, device, train_loader_pin_memory, test_loader_pin_memory)


以上
